In [1]:
import numpy as np
import pandas as pd 
import csv 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter("ignore")
import scipy.optimize as opt
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn import metrics


In [2]:
#  loading the data set 
file = pd.read_csv('BSOM_DataSet_for_HW3.csv')


# to check whether there are any empty values or not 
df=file[['all_mcqs_avg_n20', 'all_NBME_avg_n4', 'CBSE_01',  'CBSE_02', 'LEVEL']]


df['LEVEL'].fillna(value='A', inplace=True) 

df[['CBSE_02']].fillna(method ='bfill')
#.fillna(52, inplace=True)
df.fillna(df['CBSE_02'].mean(), inplace=True)

df.isnull().any()


all_mcqs_avg_n20    False
all_NBME_avg_n4     False
CBSE_01             False
CBSE_02             False
LEVEL               False
dtype: bool

In [3]:
LEVEL = {'A': 0,'B': 1,'C':2,'D':3}

df.LEVEL = [LEVEL[item] for item in df.LEVEL] 
print(df) 

     all_mcqs_avg_n20  all_NBME_avg_n4  CBSE_01  CBSE_02  LEVEL
0               0.736           0.7700       42     68.0      1
1               0.740           0.8000       44     67.0      2
2               0.807           0.8125       41     78.0      0
3               0.886           0.9250       68     91.0      0
4               0.839           0.8550       57     74.0      1
..                ...              ...      ...      ...    ...
110             0.637           0.6825       45     52.0      2
111             0.751           0.8150       53     60.0      2
112             0.811           0.8325       50     77.0      1
113             0.729           0.7275       43     68.0      2
114             0.839           0.8625       54     73.0      1

[115 rows x 5 columns]


In [4]:


from sklearn.compose import ColumnTransformer

#onhtencder = OneHotEncoder(categorical_features = [0]) 

onhtencder= ColumnTransformer([("LEVEL", OneHotEncoder(), [0])], remainder = 'passthrough')

data = onhtencder.fit_transform(df[['LEVEL']]).toarray() 
#data

In [5]:

X= df.iloc[:,:]

X_norm = (X.iloc[:,:-1] - X.iloc[:,:-1].mean()) / (X.iloc[:,:-1].max() - X.iloc[:,:-1].min())



X_modified = pd.concat([X_norm, X.iloc[:,-1]],axis=1)

target=df.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X_modified, data, test_size=0.20,random_state=9)

In [6]:

Train_ip=np.array(X_train.iloc[:,:-1])
Train_otpts=np.array(y_train)


In [7]:
def Weights(ttl_weits,ip,hidn,otpt):
    w1 = np.reshape(ttl_weits[:hidn*(ip+1)], (hidn, ip+1), 'F')
    w2 = np.reshape(ttl_weits[hidn*(ip+1):], (otpt, hidn+1), 'F')
    return w1,w2

    

In [8]:
def SIG(z):
    return 1 / (1 + np.exp(-z))


def SIGGrad(z):
    return np.multiply(SIG(z), 1-SIG(z))



In [9]:
def Cost(h,y,wt1,wt2,l,lmbda):
    cost = np.sum(np.multiply(y, np.log(h))+np.multiply(1-y, np.log(1-h)))
    s_1 = np.sum(np.sum(np.power(wt1[:,1:],2), axis = 1))
    
    s_2 = np.sum(np.sum(np.power(wt2[:,1:],2), axis = 1))
    
    re= np.sum(cost / (-l)) + (s_1 + s_2) * lmbda / (2*l)    
    
    return re

In [10]:


np.random.seed(14)

wt1 = np.random.random((1,5))    
wt2 = np.random.random((4,2))    
ttl_weits = np.hstack((wt1.ravel(order='F'), wt2.ravel(order='F')))   

# layers configuration 

ip_lyr = 4
hd_lyr = 1
no_labls = 4

print('wt1',wt1)
print('wt2',wt2)
print('ttl_weits',ttl_weits)
#print(ttl_weits.shape)

wt1 [[0.51394334 0.77316505 0.87042769 0.00804695 0.30973593]]
wt2 [[0.95760374 0.51311671]
 [0.31828442 0.53919994]
 [0.22125494 0.80648136]
 [0.34225463 0.53888885]]
ttl_weits [0.51394334 0.77316505 0.87042769 0.00804695 0.30973593 0.95760374
 0.31828442 0.22125494 0.34225463 0.51311671 0.53919994 0.80648136
 0.53888885]


In [11]:
def SIG(z):
    return 1/(1+np.exp(-z))

In [12]:
def Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, X, y, lmbda):
    wt1,wt2= Weights(ttl_weits, ip_lyr, hd_lyr, no_labls)
   
    l = len(y)
   
    ones = np.ones((l,1))
    #print('ones',ones)
    a1 = np.hstack((ones, X))
    #print('a1',a1)
    a2 = SIG(a1 @ wt1.T)
    #print('a2',a2)
    a2 = np.hstack((ones, a2))
    #print('a2',a2)
    h = SIG(a2 @ wt2.T)
    #print('h',h)

   
    
    c=Cost(h,Train_otpts,wt1,wt2,l,lmbda)
    
    

    
    return c
    

In [13]:
cf_er = Feedfrwd(ttl_weits, ip_lyr, hd_lyr, no_labls, Train_ip,Train_otpts,1)


In [14]:
def NN(ttl_weits, ip_lyr, hd_lyr, no_labls, X, y, lmbda):
    
    initt_wt1,initt_wt2= Weights(ttl_weits, ip_lyr, hd_lyr, no_labls)
    
  
    lenth = len(y)

    ones = np.ones((lenth,1))
    
    
    a1 = np.hstack((ones, X))
    #print('a1',a1)
    #print('al shape',a1.shape)
    z2 = a1 @ initt_wt1.T
    #print('z2',z2)
    #print('z2 sha',z2.shape)
    a2 = np.hstack((ones, SIG(z2)))
    #print('a2 ',a2)
    #print('a2 ',a2.shape)
    z3 = a2 @ initt_wt2.T
    #print('z3',z3)
    #print('z3 ',z3.shape)
    a3 = SIG(z3)
    #print('a3',a3)
    #print('a3',a3.shape)
    y1=pd.DataFrame(y)

    d3 = a3 - y
    #print('d3',d3)
    #print('d3',d3.shape)
    z2 = np.hstack((ones, z2))
    #print('z2',z2)
    #print('z2',z2.shape)
    d2 = np.multiply(initt_wt2.T @ d3.T, SIGGrad(z2).T)
    #print('d2',d2)
    #print('d2',d2.shape)
    delta1 = np.zeros(initt_wt1.shape)
    #print('delta1',delta1)
    #print('delta1 shape',delta1.shape)
    delta2 = np.zeros(initt_wt2.shape)
    #print('delta2',delta2)
    #print('delta2 shape',delta2.shape)
    delta1 = delta1 + d2[1:,:] @ a1
    #print('delta1',delta1)
    #print('delta1',delta1.shape)
    delta2 = delta2 + d3.T @ a2
    #print('delta2',delta2)
    #print('delta2 ',delta2.shape)
        
        
    delta1 /= lenth
    delta2 /= lenth
    #print(delta1.shape, delta2.shape)
    delta1[:,1:] = delta1[:,1:] + initt_wt1[:,1:] * lmbda / lenth
    delta2[:,1:] = delta2[:,1:] + initt_wt2[:,1:] * lmbda / lenth
        
    return np.hstack((delta1.ravel(order='F'), delta2.ravel(order='F')))

In [15]:
bckprop_wts = NN(ttl_weits, ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts, 1)



In [16]:
def predict(wt1, wt2, X, y):
    l = len(y)
    ones = np.ones((l,1))
    a1 = np.hstack((ones, X))
    a2 = SIG(a1 @ wt1.T)
    a2 = np.hstack((ones, a2))
    h = SIG(a2 @ wt2.T)
    return np.argmax(h, axis = 1) 

In [17]:

def checkGradient(frwd_wts,bckprop_wts,iplyr, hdlyr, no_labls,Train_ip,Train_otpts,lmbda):
    eps = 0.22
   
    
    for i in range(5):
        iterattion = int(np.random.rand()*len(frwd_wts))
        #print('iterattion',iterattion)
        adding_eps = np.zeros((len(frwd_wts),1))
        #print('adding_eps',adding_eps)
        adding_eps[iterattion] = eps
        #print('adding_eps[iterattion]',adding_eps[iterattion])

        ThetaPlus = Feedfrwd(frwd_wts + adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts,lmbda)
        
        #print('ThetaPlus',ThetaPlus)
        
        ThetaMinus  = Feedfrwd(frwd_wts - adding_eps.flatten(),iplyr, hdlyr, no_labls,Train_ip,Train_otpts,lmbda)
        #print('ThetaMinus',ThetaMinus)
        
        num_app = (ThetaPlus - ThetaMinus) / float(2*eps)
        print('num_app',num_app)
       
        print('bckprop_wts[iterattion]',bckprop_wts[iterattion])
        
        



In [18]:
grad_chk=checkGradient(ttl_weits,bckprop_wts,ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts, 1)
grad_chk

num_app 0.23724924495991295
bckprop_wts[iterattion] 0.2380399230891398
num_app 0.618230677532917
bckprop_wts[iterattion] 0.6188129273093341
num_app 0.00784310494301519
bckprop_wts[iterattion] 0.007837365514406388
num_app 0.39201004235539316
bckprop_wts[iterattion] 0.3921585715843909
num_app 0.0017451302613203832
bckprop_wts[iterattion] 0.0017433444037930138


## Lambda =0.01

In [19]:



Final_wts_1 = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts,0.01))


Final_wts1_1,Final_wts2_1= Weights(Final_wts_1, ip_lyr, hd_lyr, no_labls)

pred_1 = predict(Final_wts1_1, Final_wts2_1, X_test.iloc[:,:-1], y_test)


y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)

print(confusion_matrix(y_actual,pred_1))




print('f1_score',f1_score(y_actual,pred_1, average="weighted"))
print('precision_score',precision_score(y_actual,pred_1, average="weighted"))
print('recall_score',recall_score(y_actual,pred_1, average="weighted"))  




fpr_1, tpr_1, thresholds_1 = metrics.roc_curve(y_actual,pred_1, pos_label=2)
print('AUC score :',metrics.auc(fpr_1, tpr_1))



         Current function value: 1.404095
         Iterations: 100
         Function evaluations: 274
         Gradient evaluations: 274
[[4 0 1 0]
 [4 7 0 0]
 [0 4 2 0]
 [0 1 0 0]]
f1_score 0.5408366051088168
precision_score 0.5615942028985508
recall_score 0.5652173913043478
AUC score : 0.7941176470588236


## Lambda =0.5

In [20]:


Final_wts_2 = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts,0.5))


Final_wts1_2,Final_wts2_2= Weights(Final_wts_2, ip_lyr, hd_lyr, no_labls)

pred_2 = predict(Final_wts1_2, Final_wts2_2, X_test.iloc[:,:-1], y_test)


y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)

print(confusion_matrix(y_actual,pred_2))

print('f1_score',f1_score(y_actual,pred_2, average="weighted"))
print('precision_score',precision_score(y_actual,pred_2, average="weighted"))
print('recall_score',recall_score(y_actual,pred_2, average="weighted"))  
fpr_2, tpr_2, thresholds_2 = metrics.roc_curve(y_actual,pred_2, pos_label=2)
print('AUC score :',metrics.auc(fpr_2, tpr_2))



Optimization terminated successfully.
         Current function value: 1.760457
         Iterations: 38
         Function evaluations: 97
         Gradient evaluations: 97
[[4 0 1 0]
 [7 4 0 0]
 [0 5 1 0]
 [0 1 0 0]]
f1_score 0.3561076604554866
precision_score 0.4007905138339921
recall_score 0.391304347826087
AUC score : 0.8235294117647058


## Lambda =1

In [21]:
## Lambda =1



Final_wts_3 = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts,1))


Final_wts1_3,Final_wts2_3= Weights(Final_wts_3, ip_lyr, hd_lyr, no_labls)

pred_3 = predict(Final_wts1_3, Final_wts2_3, X_test.iloc[:,:-1], y_test)


y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)

print(confusion_matrix(y_actual,pred_3))

print('f1_score',f1_score(y_actual,pred_3, average="weighted"))
print('precision_score',precision_score(y_actual,pred_3, average="weighted"))
print('recall_score',recall_score(y_actual,pred_3, average="weighted"))  


fpr_3, tpr_3, thresholds_3 = metrics.roc_curve(y_actual,pred_3, pos_label=2)
print('AUC score :',metrics.auc(fpr_3, tpr_3))



Optimization terminated successfully.
         Current function value: 1.890400
         Iterations: 27
         Function evaluations: 73
         Gradient evaluations: 73
[[4 0 1 0]
 [7 4 0 0]
 [0 5 1 0]
 [0 1 0 0]]
f1_score 0.3561076604554866
precision_score 0.4007905138339921
recall_score 0.391304347826087
AUC score : 0.8235294117647058


## Lambda =5

In [22]:



Final_wts_4 = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts,5))


Final_wts1_4,Final_wts2_4= Weights(Final_wts_4, ip_lyr, hd_lyr, no_labls)

pred_4 = predict(Final_wts1_4, Final_wts2_4, X_test.iloc[:,:-1], y_test)


y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)

print(confusion_matrix(y_actual,pred_4))




print('f1_score',f1_score(y_actual,pred_4, average="weighted"))
print('precision_score',precision_score(y_actual,pred_4, average="weighted"))
print('recall_score',recall_score(y_actual,pred_4, average="weighted"))  




fpr_4, tpr_4, thresholds_4 = metrics.roc_curve(y_actual,pred_4, pos_label=2)
print('AUC score :',metrics.auc(fpr_4, tpr_4))




Optimization terminated successfully.
         Current function value: 2.040298
         Iterations: 19
         Function evaluations: 47
         Gradient evaluations: 47
[[ 0  5  0  0]
 [ 0 11  0  0]
 [ 0  6  0  0]
 [ 0  1  0  0]]
f1_score 0.309462915601023
precision_score 0.2287334593572779
recall_score 0.4782608695652174
AUC score : 0.5


## Lambda =50

In [23]:



Final_wts_5 = opt.fmin_cg(maxiter = 100, f = Feedfrwd, x0 = ttl_weits, fprime = NN,args = (ip_lyr, hd_lyr, no_labls,Train_ip,Train_otpts,50))


Final_wts1_5,Final_wts2_5= Weights(Final_wts_5, ip_lyr, hd_lyr, no_labls)

pred_5 = predict(Final_wts1_5, Final_wts2_5, X_test.iloc[:,:-1], y_test)


y_actual=X_test.iloc[:,-1]
y_actual=np.array(y_actual)

print(confusion_matrix(y_actual,pred_5))




print('f1_score',f1_score(y_actual,pred_5, average="weighted"))
print('precision_score',precision_score(y_actual,pred_5, average="weighted"))
print('recall_score',recall_score(y_actual,pred_5, average="weighted"))  




fpr_5, tpr_5, thresholds_5 = metrics.roc_curve(y_actual,pred_5, pos_label=2)
print('AUC score :',metrics.auc(fpr_5, tpr_5))



Optimization terminated successfully.
         Current function value: 2.040298
         Iterations: 19
         Function evaluations: 39
         Gradient evaluations: 39
[[ 0  5  0  0]
 [ 0 11  0  0]
 [ 0  6  0  0]
 [ 0  1  0  0]]
f1_score 0.309462915601023
precision_score 0.2287334593572779
recall_score 0.4782608695652174
AUC score : 0.5
